In [ ]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 32.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
import os
import shutil

mountpoint = '/content/drive'
if os.path.exists(mountpoint):
    print(f"Removing existing mountpoint directory: {mountpoint}")
    shutil.rmtree(mountpoint)

drive.mount(mountpoint)

Removing existing mountpoint directory: /content/drive
Mounted at /content/drive


# Testing for one tif to png

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

tif_path = "/content/drive/MyDrive/xview2_parts/geotiffs/hold/images/guatemala-volcano_00000004_post_disaster.tif"
png_output_path = "/content/drive/MyDrive/xview2_parts/output_image.png"

with rasterio.open(tif_path) as src:
    b = src.read(1)
    g = src.read(2)
    r = src.read(3)

rgb = np.stack((r, g, b), axis=-1)

def normalize(channel):
    channel = channel.astype(np.float32)
    channel = 255 * (channel - channel.min()) / (channel.max() - channel.min())
    return channel.astype(np.uint8)

rgb_norm = np.stack([normalize(rgb[..., i]) for i in range(3)], axis=-1)

Image.fromarray(rgb_norm).save(png_output_path)

print("✅ PNG image saved!")

✅ PNG image saved!


# All images for the training from tif to colorfull png

In [ ]:
import os
import rasterio
import numpy as np
from PIL import Image

tif_folder = "/content/drive/MyDrive/xview2/geotiffs/test/forAnomalyMaps/"
png_output_folder = "/content/drive/MyDrive/xview2/geotiffs/test/newToMove/"
os.makedirs(png_output_folder, exist_ok=True)

def normalize(channel):
    channel = channel.astype(np.float32)
    channel = 255 * (channel - channel.min()) / (channel.max() - channel.min() + 1e-6)
    return channel.astype(np.uint8)

for filename in os.listdir(tif_folder):
    if filename.endswith(".tif") and "pre" in filename.lower():
        tif_path = os.path.join(tif_folder, filename)
        png_filename = os.path.splitext(filename)[0] + ".png"
        png_path = os.path.join(png_output_folder, png_filename)

        try:
            with rasterio.open(tif_path) as src:
                if src.count >= 3:
                    r = src.read(3)
                    g = src.read(2)
                    b = src.read(1)

                    rgb = np.stack((r, g, b), axis=-1)
                    rgb_norm = np.stack([normalize(rgb[..., i]) for i in range(3)], axis=-1)

                    Image.fromarray(rgb_norm).save(png_path)
                    print(f"✅ Saved: {png_filename}")

                    # Delete original .tif
                    os.remove(tif_path)
                    print(f"🗑️ Deleted: {filename}")
                else:
                    print(f"⚠️ Skipped (not enough bands): {filename}")
        except Exception as e:
            print(f"❌ Error processing {filename}: {e}")


# 150 images to be made for test

In [ ]:
import os
import rasterio
import numpy as np
from PIL import Image

# Paths
tif_folder = "/content/drive/MyDrive/xview2/geotiffs/test/forAnomalyMaps/"
png_output_folder = "/content/drive/MyDrive/xview2/geotiffs/test/newToMove/"
os.makedirs(png_output_folder, exist_ok=True)

def normalize(channel):
    channel = channel.astype(np.float32)
    channel = 255 * (channel - channel.min()) / (channel.max() - channel.min() + 1e-6)
    return channel.astype(np.uint8)

post_tifs = [f for f in os.listdir(tif_folder) if f.endswith(".tif") and "post" in f.lower()]
post_tifs = sorted(post_tifs)[:100]

for filename in post_tifs:
    tif_path = os.path.join(tif_folder, filename)
    png_filename = os.path.splitext(filename)[0] + ".png"
    png_path = os.path.join(png_output_folder, png_filename)

    try:
        with rasterio.open(tif_path) as src:
            if src.count >= 3:
                r = src.read(3)
                g = src.read(2)
                b = src.read(1)

                rgb = np.stack((r, g, b), axis=-1)
                rgb_norm = np.stack([normalize(rgb[..., i]) for i in range(3)], axis=-1)

                Image.fromarray(rgb_norm).save(png_path)
                print(f"✅ Saved: {png_filename}")

                os.remove(tif_path)
                print(f"🗑️ Deleted: {filename}")
            else:
                print(f"⚠️ Skipped (not enough bands): {filename}")
    except Exception as e:
        print(f"❌ Error processing {filename}: {e}")


✅ Saved: midwest-flooding_00000292_post_disaster.png
🗑️ Deleted: midwest-flooding_00000292_post_disaster.tif
✅ Saved: midwest-flooding_00000298_post_disaster.png
🗑️ Deleted: midwest-flooding_00000298_post_disaster.tif
✅ Saved: palu-tsunami_00000041_post_disaster.png
🗑️ Deleted: palu-tsunami_00000041_post_disaster.tif
✅ Saved: palu-tsunami_00000081_post_disaster.png
🗑️ Deleted: palu-tsunami_00000081_post_disaster.tif
✅ Saved: santa-rosa-wildfire_00000087_post_disaster.png
🗑️ Deleted: santa-rosa-wildfire_00000087_post_disaster.tif
✅ Saved: santa-rosa-wildfire_00000099_post_disaster.png
🗑️ Deleted: santa-rosa-wildfire_00000099_post_disaster.tif


# To see how many images are in the path

In [ ]:
png_output_folder = "/content/drive/MyDrive/xview2_parts/train/good/"

total_files = len(os.listdir(png_output_folder))
print(f"📁 Total files in folder: {total_files}")

📁 Total files in folder: 933


# Moves 150 pngs to the good for test from good from train

In [ ]:
import os
import shutil

source_folder = "/content/drive/MyDrive/xview2_parts/train/good/"
destination_folder = "/content/drive/MyDrive/xview2_parts/test/good/"

# Make sure the destination folder exists
os.makedirs(destination_folder, exist_ok=True)

# Get list of all PNG files
png_files = [f for f in os.listdir(source_folder) if f.endswith(".png")]

# Sort and select the first 150
png_files_to_move = sorted(png_files)[:150]

# Move the files
for filename in png_files_to_move:
    src_path = os.path.join(source_folder, filename)
    dst_path = os.path.join(destination_folder, filename)
    shutil.move(src_path, dst_path)
    print(f"✅ Moved: {filename}")

print(f"\n✅ Done! {len(png_files_to_move)} images moved.")


✅ Moved: guatemala-volcano_00000004_pre_disaster.png
✅ Moved: guatemala-volcano_00000012_pre_disaster.png
✅ Moved: guatemala-volcano_00000014_pre_disaster.png
✅ Moved: guatemala-volcano_00000020_pre_disaster.png
✅ Moved: guatemala-volcano_00000022_pre_disaster.png
✅ Moved: hurricane-florence_00000006_pre_disaster.png
✅ Moved: hurricane-florence_00000009_pre_disaster.png
✅ Moved: hurricane-florence_00000010_pre_disaster.png
✅ Moved: hurricane-florence_00000011_pre_disaster.png
✅ Moved: hurricane-florence_00000014_pre_disaster.png
✅ Moved: hurricane-florence_00000017_pre_disaster.png
✅ Moved: hurricane-florence_00000021_pre_disaster.png
✅ Moved: hurricane-florence_00000025_pre_disaster.png
✅ Moved: hurricane-florence_00000033_pre_disaster.png
✅ Moved: hurricane-florence_00000034_pre_disaster.png
✅ Moved: hurricane-florence_00000035_pre_disaster.png
✅ Moved: hurricane-florence_00000038_pre_disaster.png
✅ Moved: hurricane-florence_00000040_pre_disaster.png
✅ Moved: hurricane-florence_00000